# Replication Status

This notebook provides a lightweight, reproducible overview of the current replication status.
Replication objective: reproduce the paper's econometric results in Python with Stata 8/9-compatible defaults.
It demonstrates the model pipeline on synthetic data only.

Project docs: [README.md](../README.md) and [replication-plan.md](../replication-plan.md).


In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display
from linearmodels.panel import PanelOLS
from linearmodels.iv import IV2SLS

print(sys.version)


In [ ]:
# Synthetic data generation (deterministic)
rng = np.random.default_rng(42)

years = list(range(2001, 2011))
countries = [
    ("USA", "CONCACAF", 35.0, 285.0, 25.0, 2.0, 0.5, 77.0, 81.0, 40.0),
    ("CAN", "CONCACAF", 32.0, 31.0, 60.0, 1.8, 0.3, 79.0, 80.0, 55.0),
    ("MEX", "CONCACAF", 8.0, 100.0, 55.0, 4.0, 1.2, 74.0, 76.0, 35.0),
    ("BRA", "CONMEBOL", 7.5, 175.0, 25.0, 6.0, 2.5, 72.0, 84.0, 10.0),
    ("ARG", "CONMEBOL", 9.0, 37.0, 35.0, 5.0, 2.0, 73.0, 91.0, 12.0),
    ("GER", "UEFA", 30.0, 82.0, 65.0, 2.0, 0.2, 78.0, 74.0, 8.0),
    ("ENG", "UEFA", 28.0, 59.0, 55.0, 2.3, 0.8, 78.5, 90.0, 6.0),
    ("ESP", "UEFA", 22.0, 40.0, 50.0, 2.7, 0.3, 79.0, 77.0, 7.0),
    ("JPN", "AFC", 26.0, 125.0, 30.0, 0.5, 0.1, 81.5, 66.0, 25.0),
    ("KOR", "AFC", 18.0, 47.0, 70.0, 2.5, 0.1, 78.5, 80.0, 28.0),
    ("NGA", "CAF", 2.5, 120.0, 40.0, 9.0, 15.0, 51.0, 45.0, 60.0),
    ("EGY", "CAF", 3.0, 70.0, 50.0, 7.0, 5.0, 70.0, 43.0, 50.0),
]

rows = []
for country, confed, gdp0, pop0, trade0, infl0, oil0, leb0, urb0, club0 in countries:
    for year in years:
        t = year - years[0]
        gdp = gdp0 + 0.6 * t + rng.uniform(-0.6, 0.6)
        pop = pop0 + 0.7 * t + rng.uniform(-0.4, 0.4)
        trade = trade0 + 0.2 * t + rng.uniform(-1.0, 1.0)
        infl = max(0.1, infl0 + 0.1 * t + rng.uniform(-0.3, 0.3))
        oil = max(0.0, oil0 + rng.uniform(-0.2, 0.2))
        leb = leb0 + 0.05 * t + rng.uniform(-0.2, 0.2)
        urb = urb0 + 0.08 * t + rng.uniform(-0.3, 0.3)
        club = max(1.0, club0 - 0.05 * urb + 0.02 * gdp + rng.uniform(-2.0, 2.0))
        fifa = (
            900
            + 6.0 * gdp
            + 0.7 * pop
            + 1.5 * trade
            - 4.0 * infl
            - 0.2 * oil
            + 4.0 * leb
            - 1.1 * club
            + rng.uniform(-15.0, 15.0)
        )
        rows.append({
            "country": country,
            "year": year,
            "confed": confed,
            "fifa_points": fifa,
            "gdp_pc": gdp,
            "gdp_pc_sq": gdp ** 2,
            "pop": pop,
            "pop_sq": pop ** 2,
            "trade": trade,
            "infl": infl,
            "oil": oil,
            "leb": leb,
            "club": club,
            "urbpop": urb,
            "urbpop_sq": urb ** 2,
        })

df = pd.DataFrame(rows)


## Synthetic data snapshot
This is illustrative only and does not represent the paper's real data.


In [ ]:
df.head()
print(df.shape)


In [ ]:
# Load model runner helpers (preferred)
repo_root = Path.cwd().resolve()
candidate = repo_root
found_root = None
for _ in range(6):
    if (candidate / "scripts" / "replicate_stata.py").exists():
        found_root = candidate
        break
    if candidate.parent == candidate:
        break
    candidate = candidate.parent
if found_root:
    sys.path.append(str(found_root))

try:
    from scripts.replicate_stata import fit_fe_ols, fit_fe_iv
except Exception:
    # Fallback: minimal local copies
    def two_way_demean(df, cols, entity, time):
        out = df.copy()
        for col in cols:
            overall = out[col].mean()
            ent_mean = out.groupby(entity)[col].transform("mean")
            time_mean = out.groupby(time)[col].transform("mean")
            out[col] = out[col] - ent_mean - time_mean + overall
        return out

    def _drop_absorbed(panel, exog):
        names = list(panel.index.names)
        if len(names) < 2 or names[0] is None or names[1] is None:
            return exog, []

        entity, time = names[0], names[1]
        work = panel.reset_index()
        demeaned = two_way_demean(work, exog, entity, time)
        dropped = []
        keep = []
        for col in exog:
            if demeaned[col].var() <= 1e-12:
                dropped.append(col)
            else:
                keep.append(col)
        return keep, dropped

    def fit_fe_ols(panel, dep, exog):
        keep, dropped = _drop_absorbed(panel, exog)
        if dropped:
            print("Dropped absorbed variables: {0}".format(", ".join(dropped)))
        if not keep:
            raise ValueError("All regressors absorbed by fixed effects.")

        y = panel[dep]
        X = panel[keep]
        model = PanelOLS(y, X, entity_effects=True, time_effects=True)
        return model.fit(cov_type="clustered", cluster_entity=True, debiased=True)

    def fit_fe_iv(df, dep, exog, endog, instr, entity, time):
        cols = [dep] + exog + [endog] + instr + [entity, time]
        work = df[cols].dropna().copy()
        work = two_way_demean(work, [dep] + exog + [endog] + instr, entity, time)

        y = work[dep]
        keep_exog = [c for c in exog if work[c].var() > 1e-12]
        if not keep_exog:
            raise ValueError("All exogenous regressors absorbed by fixed effects.")

        endog_v = work[endog]
        if endog_v.var() <= 1e-12:
            raise ValueError("Endogenous regressor absorbed by fixed effects.")

        keep_instr = [c for c in instr if work[c].var() > 1e-12]
        if not keep_instr:
            raise ValueError("All instruments absorbed by fixed effects.")

        X = work[keep_exog]
        Z = work[keep_instr]

        model = IV2SLS(y, X, endog_v, Z)
        return model.fit(cov_type="clustered", clusters=work[entity], debiased=True)


In [ ]:
dep = "fifa_points"
base = ["gdp_pc", "gdp_pc_sq", "pop", "pop_sq"]
macro = ["trade", "infl", "oil", "leb"]
club = "club"
instr = ["urbpop", "urbpop_sq"]

panel = df.set_index(["country", "year"])
res1 = fit_fe_ols(panel, dep, base)
display(res1.summary)


In [ ]:
res2 = fit_fe_ols(panel, dep, base + macro)
display(res2.summary)


In [ ]:
res3 = fit_fe_iv(df, dep, base + macro, club, instr, "country", "year")
display(res3.summary)


In [ ]:
# Write outputs to results/ (synthetic only)
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

def write_summary(res, path):
    table = res.summary.tables[1].as_csv()
    path.write_text(table)

write_summary(res1, results_dir / "eq1_full.csv")
write_summary(res2, results_dir / "eq2_full.csv")
write_summary(res3, results_dir / "eq3_full.csv")


## Replication status
- [x] Synthetic run
- [ ] Real data ingest
- [ ] Stata comparison
- [ ] Output format parity
- [ ] Validation and tolerances


## Next steps
- Fill out [docs/data-dictionary.md](../docs/data-dictionary.md).
- Review [replication-plan.md](../replication-plan.md) for open questions.
- Confirm the Stata version used for the original runs.
